[Reference](https://boadziedaniel.medium.com/building-real-time-dashboards-with-fastapi-and-htmx-01ea458673cb)

In [1]:
!uv init sensor-dashboard
!cd sensor-dashboard
!uv add fastapi uvicorn sse-starlette jinja2 python-multipart arel

Initialized project `sensor-dashboard` at `/content/sensor-dashboard`
error: No `pyproject.toml` found in current directory or any parent directory


```
sensor-dashboard/
├── app.py              # The brain
├── utils.py            # Fake sensor magic
├── templates/          # HTML that doesn't suck
│   ├── base.html
│   ├── index.html
│   └── chart_data.html
└── pyproject.toml      # uv's config
```

In [2]:
from collections import deque
import random
from datetime import datetime

class SensorData:
    def __init__(self):
        # Room temperature range (adjust if you live in Antarctica)
        self.min_temp = 18.0
        self.max_temp = 26.0
        self.min_humidity = 30.0
        self.max_humidity = 65.0

    def generate_reading(self):
        return {
            "timestamp": datetime.now().isoformat(),
            "temperature": round(random.uniform(self.min_temp, self.max_temp), 1),
            "humidity": round(random.uniform(self.min_humidity, self.max_humidity), 1),
            "status": random.choice(["normal", "warning", "critical"])
        }
# Store the last 20 readings (because nobody cares about data from 1995)
recent_readings = deque(maxlen=20)

In [4]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from sse_starlette import EventSourceResponse
from utils import SensorData, recent_readings
import json
import asyncio
import arel
import os

app = FastAPI(debug=True)
templates = Jinja2Templates(directory="templates")
sensor = SensorData()

# Hot reload magic for development (because restarting servers is for losers)
if os.getenv("DEBUG"):
    hot_reload = arel.HotReload(paths=["."])
    app.add_websocket_route("/hot-reload", route=hot_reload)
    app.add_event_handler("startup", hot_reload.startup)
    app.add_event_handler("shutdown", hot_reload.shutdown)
    templates.env.globals["DEBUG"] = True
    templates.env.globals["hot_reload"] = hot_reload

@app.get("/", response_class=HTMLResponse)
def dashboard(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.get("/stream")
async def stream_sensor_data():
    """The magic streaming endpoint that makes everything work"""
    async def event_generator():
        try:
            while True:
                # Generate new sensor reading
                data = sensor.generate_reading()
                recent_readings.append(data)  # Store for charts

                # Send to all connected browsers
                yield {
                    "event": "sensor_update",
                    "data": json.dumps(data)
                }
                await asyncio.sleep(2)  # Update every 2 seconds
        except asyncio.CancelledError:
            # User closed browser/tab - no drama, just stop
            pass

    return EventSourceResponse(event_generator())

@app.get("/chart-data")
async def get_chart_data(request: Request):
    """Prepare data for Chart.js visualization"""
    # Ensure we have enough data for charts
    if len(recent_readings) < 20:
        for _ in range(20):
            recent_readings.append(sensor.generate_reading())

    temp_data = [r['temperature'] for r in recent_readings]
    humidity_data = [r['humidity'] for r in recent_readings]
    labels = [str(i) for i in range(len(recent_readings))]

    return templates.TemplateResponse("chart_data.html", {
        "request": request,
        "temp_data": json.dumps(temp_data),
        "humidity_data": json.dumps(humidity_data),
        "labels": json.dumps(labels)
    })


@app.get("/health")
async def health_check():
    """Because production servers need to know we're alive"""
    return {"status": "alive_and_kicking", "timestamp": datetime.now().isoformat()}

templates/base.html

```
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Sensor Dashboard{% endblock %}</title>
    
    <!-- HTMX - The star of the show -->
    <script src="https://unpkg.com/htmx.org@1.9.10/dist/htmx.min.js"></script>
    <script src="https://unpkg.com/htmx.org@1.9.10/dist/ext/sse.js"></script>
    
    <!-- DaisyUI + Tailwind - Because life's too short for ugly UIs -->
    <link href="https://cdn.jsdelivr.net/npm/daisyui@4.6.0/dist/full.min.css" rel="stylesheet">
    <script src="https://cdn.tailwindcss.com"></script>
    
    {% if DEBUG %}
    <!-- Hot reload for development -->
    <script>
        new EventSource('/hot-reload').onmessage = () => location.reload()
    </script>
    {% endif %}
</head>
<body class="bg-base-200 min-h-screen">
    {% block content %}{% endblock %}
</body>
</html>
```

templates/index.html

```
{% extends 'base.html' %}
{% block content %}
<div class="container mx-auto px-4 py-4 max-w-7xl">
    <!-- Header with connection status -->
    <div class="flex flex-col sm:flex-row sm:items-center sm:justify-between mb-6 gap-4">
        <h1 class="text-3xl sm:text-4xl font-bold bg-gradient-to-r from-primary to-secondary bg-clip-text text-transparent">
            Environmental Dashboard
        </h1>
        <div class="flex items-center gap-2">
            <div class="loading loading-ring loading-sm text-success" id="connection-indicator"></div>
            <span class="text-sm" id="connection-status">Connecting...</span>
        </div>
    </div>
<!-- Live sensor data (the magic happens here) -->
    <div class="card bg-base-100 shadow-xl mb-6">
        <div class="card-body">
            <div class="flex items-center gap-2 mb-4">
                <div class="badge badge-primary">Live</div>
                <h2 class="card-title">Sensor Readings</h2>
            </div>
            
            <!-- HTMX SSE connection - this is where the magic happens -->
            <div hx-ext="sse" sse-connect="/stream">
                <div id="sensor-display" sse-swap="sensor_update" hx-swap="innerHTML">
                    <div class="flex items-center justify-center py-8">
                        <div class="loading loading-spinner loading-lg text-primary"></div>
                        <span class="ml-4">Connecting to sensors...</span>
                    </div>
                </div>
            </div>
        </div>
    </div>
    <!-- Charts side by side -->
    <div class="grid grid-cols-1 sm:grid-cols-2 gap-4">
        <div class="card bg-base-100 shadow-xl">
            <div class="card-body">
                <h3 class="card-title">Temperature Trend</h3>
                <div class="h-48">
                    <canvas id="tempChart"></canvas>
                </div>
            </div>
        </div>
        
        <div class="card bg-base-100 shadow-xl">
            <div class="card-body">
                <h3 class="card-title">Humidity Trend</h3>
                <div class="h-48">
                    <canvas id="humidityChart"></canvas>
                </div>
            </div>
        </div>
    </div>
    <!-- Hidden chart data updater -->
    <div hx-get="/chart-data" hx-trigger="sse:sensor_update" hx-target="#chart-data" hx-swap="outerHTML">
        <div id="chart-data"></div>
    </div>
</div>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<script>
let tempChart, humidityChart;
// Handle real-time sensor updates
document.body.addEventListener('htmx:sseMessage', function(e) {
    if (e.detail.type === 'sensor_update') {
        const data = JSON.parse(e.detail.data);
        
        // Update the display with new data
        const statusClass = data.status === 'normal' ? 'badge-success' :
                           data.status === 'warning' ? 'badge-warning' : 'badge-error';
        
        const html = `
            <div class="grid grid-cols-1 sm:grid-cols-2 gap-4 mb-4">
                <div class="stat bg-primary/10 rounded-lg p-4">
                    <div class="stat-title">Temperature</div>
                    <div class="stat-value text-primary">${data.temperature}°C</div>
                    <div class="stat-desc">Real-time reading</div>
                </div>
                <div class="stat bg-secondary/10 rounded-lg p-4">
                    <div class="stat-title">Humidity</div>
                    <div class="stat-value text-secondary">${data.humidity}%</div>
                    <div class="stat-desc">Relative humidity</div>
                </div>
            </div>
            <div class="flex flex-col sm:flex-row sm:justify-between sm:items-center gap-2">
                <div class="badge ${statusClass} text-white font-bold">
                    ${data.status.toUpperCase()}
                </div>
                <div class="text-sm text-base-content/60">
                    ${new Date(data.timestamp).toLocaleTimeString()}
                </div>
            </div>
        `;
        
        document.getElementById('sensor-display').innerHTML = html;
        
        // Show critical alerts (because drama is important)
        if (data.status === 'critical') {
            showCriticalAlert(data);
        }
    }
});
function showCriticalAlert(data) {
    const alertHtml = `
        <div role="alert" class="alert alert-error text-white mb-4" id="critical-alert">
            <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" class="h-6 w-6 shrink-0 stroke-current">
                <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M13 16h-1v-4h-1m1-4h.01M21 12a9 9 0 11-18 0 9 9 0 0118 0z"></path>
            </svg>
            <span><strong>🚨 Critical Alert:</strong> Temp ${data.temperature}°C, Humidity ${data.humidity}%</span>
        </div>
    `;
    
    if (!document.getElementById('critical-alert')) {
        document.querySelector('.container').insertAdjacentHTML('afterbegin', alertHtml);
        setTimeout(() => {
            document.getElementById('critical-alert')?.remove();
        }, 10000); // Auto-dismiss after 10 seconds
    }
}
// Update charts when new data arrives
document.body.addEventListener('htmx:afterSwap', function(e) {
    if (e.target.id === 'chart-data') {
        const dataElement = e.target.querySelector('[data-temp]') || e.target;
        
        if (dataElement.dataset.temp) {
            const tempData = JSON.parse(dataElement.dataset.temp);
            const humidityData = JSON.parse(dataElement.dataset.humidity);
            const labels = JSON.parse(dataElement.dataset.labels);
            
            if (tempChart && humidityChart) {
                // Update existing charts
                tempChart.data.datasets[0].data = tempData;
                tempChart.data.labels = labels;
                tempChart.update('none'); // No animation for smooth updates
                
                humidityChart.data.datasets[0].data = humidityData;
                humidityChart.data.labels = labels;
                humidityChart.update('none');
            } else {
                // Create charts for the first time
                initCharts(tempData, humidityData, labels);
            }
        }
    }
});
function initCharts(tempData, humidityData, labels) {
    const tempCtx = document.getElementById('tempChart').getContext('2d');
    tempChart = new Chart(tempCtx, {
        type: 'line',
        data: {
            labels: labels,
            datasets: [{
                data: tempData,
                borderColor: 'rgb(99, 102, 241)',
                backgroundColor: 'rgba(99, 102, 241, 0.1)',
                borderWidth: 2,
                fill: true,
                tension: 0.4
            }]
        },
        options: {
            responsive: true,
            maintainAspectRatio: false,
            plugins: { legend: { display: false } },
            scales: {
                y: { beginAtZero: false },
                x: { display: false }
            }
        }
    });
    
    const humidityCtx = document.getElementById('humidityChart').getContext('2d');
    humidityChart = new Chart(humidityCtx, {
        type: 'line',
        data: {
            labels: labels,
            datasets: [{
                data: humidityData,
                borderColor: 'rgb(16, 185, 129)',
                backgroundColor: 'rgba(16, 185, 129, 0.1)',
                borderWidth: 2,
                fill: true,
                tension: 0.4
            }]
        },
        options: {
            responsive: true,
            maintainAspectRatio: false,
            plugins: { legend: { display: false } },
            scales: {
                y: { beginAtZero: false },
                x: { display: false }
            }
        }
    });
}
// Connection status handling
document.body.addEventListener('htmx:sseOpen', function(e) {
    document.getElementById('connection-indicator').className = 'loading loading-ring loading-sm text-success';
    document.getElementById('connection-status').textContent = 'Connected';
});
document.body.addEventListener('htmx:sseError', function(e) {
    document.getElementById('connection-indicator').className = 'loading loading-ring loading-sm text-error';
    document.getElementById('connection-status').textContent = 'Connection Error';
});
</script>
{% endblock %}
```

templates/chart_data.html

```
<div id="chart-data"
     data-temp="{{ temp_data }}"
     data-humidity="{{ humidity_data }}"
     data-labels="{{ labels }}">
</div>
```

```python
# Development mode with hot reload (because life's too short for manual restarts)
DEBUG=1 uv run uvicorn app:app --reload --port 8000
```